In [1]:
import random
import operator
import pandas as pd
from collections import Counter
import nltk
from joblib import Parallel, delayed
import tqdm
import string
import numpy as np
from joblib import Parallel, delayed
import re

In [2]:
path_to_data = "./"

In [3]:
training = pd.read_csv(path_to_data + 'training_set.csv', sep=',', header=0)
training_info = pd.read_csv(path_to_data + 'training_info.csv', sep=',', header=0)
test = pd.read_csv(path_to_data + 'test_set.csv', sep=',', header=0)
test_info = pd.read_csv(path_to_data + 'test_info.csv', sep=',', header=0)

In [4]:
# training.describe()
# training.columns
# training.info
# training_info.columns
# training_info.info

In [5]:

demo = np.arange(12).reshape((3,4))
l = np.array([1,2,3])
i = np.array([1,2])
l[i]


array([2, 3])

In [9]:
demo[np.array([0,2]),]

array([[ 0,  1,  2,  3],
       [ 8,  9, 10, 11]])

In [7]:
demo[[1,],]

array([[4, 5, 6, 7]])

In [5]:
# assert all items in train_info are uniquen, in other words, there is no duplicate.
assert (len(set(training_info['mid'])) == len(training_info['mid']))
print "How many data point that we have in our training dataset:", len(training_info['mid'])

How many data point that we have in our training dataset: 43613


In [ ]:
# remove dashes and apostrophes from punctuation marks
punct = string.punctuation.replace('-', '').replace("'",'')
stpwds = nltk.corpus.stopwords.words('english')

# regex to match intra-word dashes and intra-word apostrophes
my_regex = re.compile(r"(\b[-']\b)|[\W_]")

# performs basic pre-processing
# note: we do not lowercase for consistency with Google News embeddings
def clean_string(string, punct=punct, my_regex=my_regex):
    # remove formatting
    str = re.sub('\s+', ' ', string)
    # remove punctuation (preserving dashes)
    str = ''.join(l for l in str if l not in punct)
    # remove dashes that are not intra-word
    str = my_regex.sub(lambda x: (x.group(1) if x.group(1) else ' '), str)
    # strip extra white space
    str = re.sub(' +',' ',str)
    # strip leading and trailing white space
    str = str.strip()
    # tokenize (split based on whitespace)
    tokens = str.split(' ')
    # remove stopwords
    tokens = [token for token in tokens if token not in stpwds]
    # remove tokens less than 2 characters in size
    tokens = [token.lower() for token in tokens if len(token)>=2]
    return tokens

def stemmering_sentences_messages(X):
    sentences_stem = Parallel(n_jobs=1)(delayed(clean_string)(sentence) for sentence in tqdm.tqdm(X, desc="clean sentences"))
    print type(sentences_stem)
    return sentences_stem


In [ ]:

training_info['body'] = stemmering_sentences_messages(training_info['body'])

In [8]:
test_info['body'] = stemmering_sentences_messages(test_info['body'])

clean sentences: 100%|██████████| 2362/2362 [00:02<00:00, 965.59it/s]


In [0]:
# test_info['body'][0]

In [10]:
X = training_info['body'].tolist() + test_info['body'].tolist()

In [11]:
import multiprocessing
from gensim.models.word2vec import Word2Vec
# 4 in pc
cpu_count = multiprocessing.cpu_count()
# the dimensionality of the feature vectors.
vocab_dim = 50
# number of iterations (epochs) over the corpus. Default is 5.
n_iterations = 1  # ideally more..
maxlen = 400
# ignore all words with total frequency lower than this. 
n_exposures = 30
# the maximum distance between the current and predicted word within a sentence.
window_size = 5

In [12]:
print('Training a Word2vec model...')
model = Word2Vec(size=vocab_dim,
                 min_count=n_exposures,
                 window=window_size,
                 workers=cpu_count,
                 iter=n_iterations)
model.build_vocab(X)
model.train(X)

Training a Word2vec model...


4519852

In [13]:
from gensim.corpora.dictionary import Dictionary
gensim_dict = Dictionary()
gensim_dict.doc2bow(model.vocab.keys(), allow_update=True)
# gensim_dict.items() returns [(0, u"'surpris"), (1, u'woodi'), (2, u'yellow'),...]
# K+1 aims at avoiding 0 as index.
w2indx = {v: k+1 for k, v in gensim_dict.items()}
w2vec = {word: model[word] for word in w2indx.keys()}

In [14]:
# from keras.preprocessing import sequence

In [6]:
################################
# create some handy structures #
################################
# convert training set to dictionary
emails_ids_per_sender = {}
for index, series in training.iterrows():
    row = series.tolist()
    sender = row[0]
    ids = row[1:][0].split(' ')
    emails_ids_per_sender[sender] = ids
# save all unique sender names
all_senders = emails_ids_per_sender.keys()
# create address book with frequency information for each user
address_books = {}
address_books_messages = {}
i = 0
for sender, ids in emails_ids_per_sender.iteritems():
    # recs_temp = []
    rec2mids = {}
    for my_id in ids:
        recipients = training_info[training_info['mid']==int(my_id)]['recipients'].tolist()
        recipients = recipients[0].split(' ')
        # keep only legitimate email addresses
        recipients = [rec for rec in recipients if '@' in rec]
        for rec in recipients:
            if rec in rec2mids:
                rec2mids[rec].append(my_id)
            else:
                rec2mids[rec] = [my_id]
        # recs_temp.append(recipients)
    # flatten
    # recs_temp = [elt for sublist in recs_temp for elt in sublist]
    # compute recipient counts
    # rec_occ = dict(Counter(recs_temp))
    # order by frequency
    # sorted_rec_occ = sorted(rec_occ.items(), key=operator.itemgetter(1), reverse = True)
    # save
    address_books_messages[sender] = rec2mids
    # address_books[sender] = sorted_rec_occ
    if i % 20 == 0:
        print "For the moment, we have already done", i, " lines."
    i += 1

For the moment, we have already done 0  lines.
For the moment, we have already done 20  lines.
For the moment, we have already done 40  lines.
For the moment, we have already done 60  lines.
For the moment, we have already done 80  lines.
For the moment, we have already done 100  lines.
For the moment, we have already done 120  lines.


In [16]:
# print address_books.values()[0]
# save all unique recipient names
all_recs = list(set([elt for subdict in address_books_messages.values() for elt in subdict.keys()]))
# save all unique user names
all_users = []
all_users.extend(all_senders)
all_users.extend(all_recs)
all_users = list(set(all_users))


In [17]:
print "the number of all users:", len(all_users)
print "the number of all unique recipients", len(all_recs)

the number of all users: 9783
the number of all unique recipients 9779


In [0]:
def get_k_most(k, address_books_messages, sender, id_predict, alpha):
    test_body = test_info[test_info['mid'] == int(id_predict)]['body'].tolist()[0]
    k_most = []
    count = 0
    rec_list = []
    simil_freq_list = []
    for k_rec, v_mids_list in address_books_messages[sender].iteritems():
        similarity = np.min([model.wmdistance(test_body, training_info[training_info['mid'] == int(mid)]['body'].tolist()[0]) for mid in v_mids_list])
        rec_list.append(k_rec)
        simil_freq_list.append([similarity, len(v_mids_list)])
    simil_freq_list = np.array(simil_freq_list)
    rec_list = np.array(rec_list)
    simil_freq_list[:,0] = (np.max(simil_freq_list[:,0]) - simil_freq_list[:,0]) / (np.max(simil_freq_list[:,0]) - np.min(simil_freq_list[:,0]))
    simil_freq_list[:,-1] = (simil_freq_list[:,-1] - np.min(simil_freq_list[:,-1])) / (np.max(simil_freq_list[:,-1])-np.min(simil_freq_list[:,-1]))
    socre = alpha * simil_freq_list[:,0] + (1-alpha)*simil_freq_list[:,-1]
    return simil_freq_list[np.argsort(socre)[::-1]][:k].tolist()


In [0]:
k = 10
alpha = 1/2.0
def predict_each_test_line(row):
    sender = row[0]
    # get IDs of the emails for which recipient prediction is needed
    ids_predict = row[1].split(' ')
    ids_predict = [int(my_id) for my_id in ids_predict]
    random_preds = []
    freq_preds = []
    # select k most frequent recipients for the user
    # k_most = [elt[0] for elt in address_books[sender][:k]]
    for id_predict in ids_predict:
        # select k users at random
        random_preds.append(random.sample(all_users, k))
        # for the frequency baseline, the predictions are always the same
        k_most = get_k_most(k, address_books_messages, sender, id_predict, alpha)
        freq_preds.append(k_most)
    # predictions_per_sender[sender] = [ids_predict,random_preds,freq_preds]
    return (sender, [ids_predict,random_preds,freq_preds])

In [0]:
def speedup(X):
    predictions_per_sender = Parallel(n_jobs=4)(delayed(predict_each_test_line)(row) for row in tqdm.tqdm(X, desc="speed up test"))
    return predictions_per_sender

In [ ]:
#############
# baselines #
#############
# will contain email ids, predictions for random baseline, and predictions for frequency baseline
predictions_per_sender = {}
# number of recipients to predict
k = 10
for row in test.values:
    sender = row[0]
    # get IDs of the emails for which recipient prediction is needed
    ids_predict = row[1].split(' ')
    ids_predict = [int(my_id) for my_id in ids_predict]
    random_preds = []
    freq_preds = []
    # select k most frequent recipients for the user
    # k_most = [elt[0] for elt in address_books[sender][:k]]
    for id_predict in ids_predict:
        # select k users at random
        random_preds.append(random.sample(all_users, k))
        # for the frequency baseline, the predictions are always the same
        k_most = get_k_most(k, address_books_messages, sender, id_predict, 1/2.0)
        freq_preds.append(k_most)
    predictions_per_sender[sender] = [ids_predict,random_preds,freq_preds]


In [20]:
#################################################
# write predictions in proper format for Kaggle #
#################################################

path_to_results = "/Users/zacharie/Documents/AICCours/MVA/textGraphData/"

with open(path_to_results + 'predictions_random.txt', 'wb') as my_file:
    my_file.write('mid,recipients' + '\n')
    for sender, preds in predictions_per_sender.iteritems():
        ids = preds[0]
        random_preds = preds[1]
        for index, my_preds in enumerate(random_preds):
            my_file.write(str(ids[index]) + ',' + ' '.join(my_preds) + '\n')

with open(path_to_results + 'predictions_frequency.txt', 'wb') as my_file:
    my_file.write('mid,recipients' + '\n')
    for sender, preds in predictions_per_sender.iteritems():
        ids = preds[0]
        freq_preds = preds[2]
        for index, my_preds in enumerate(freq_preds):
            my_file.write(str(ids[index]) + ',' + ' '.join(my_preds) + '\n')